<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [34]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [35]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [36]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [37]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [38]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [39]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,0.927468,0.999962,-0.600436,1.390267,0.111744,2.726109,0.642466,0.333934,2.298606
1,-0.907413,0.999929,0.593364,-0.362128,0.546883,16.521043,2.386961,0.721129,-13.629427
2,-0.787505,1.000000,-0.106745,1.132473,0.535319,1.101290,-0.617095,0.714224,-0.788428
3,0.367708,0.999991,-0.180879,2.214776,-0.359150,8.750406,0.100676,0.592856,2.925107
4,-0.113440,0.997203,0.982974,3.139334,0.232599,0.041700,-0.012171,0.480112,-0.004311
...,...,...,...,...,...,...,...,...,...
95,0.596547,0.990093,-0.894177,32.184465,0.289952,-0.150859,0.014316,0.534702,-0.082557
96,0.970966,0.989972,-0.996019,3.866204,0.593700,-0.573110,0.247469,0.747952,-0.510537
97,0.193689,0.998328,-0.203603,0.493438,0.185581,1.484246,0.172752,0.429555,0.261745
98,0.972753,0.999997,0.999851,3.068410,0.020678,2.438083,0.312673,0.143792,2.156055


In [40]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,0.347837,0.999999,0.198042,3.882749,0.279457,1.649269,0.052835,0.525199,0.521526
1,0.899839,0.986653,-0.994704,-1.472557,0.527045,6.792139,-0.579662,0.709212,5.624451
2,-0.157316,0.988257,0.937629,3.117864,-0.544815,0.649714,-0.020012,0.719903,-0.093921
3,0.830017,0.998927,-0.956903,-0.309289,0.399422,-1.621410,-2.444926,0.623607,-1.224646
4,0.613092,0.999921,-0.917253,4.319904,-0.473218,14.996356,0.111126,0.675095,8.358919
...,...,...,...,...,...,...,...,...,...
95,-0.447428,0.999703,-0.158759,3.184662,-0.451630,1.362147,-0.093977,0.660630,-0.554206
96,0.984184,0.978183,0.819624,4.399989,-0.446203,0.570193,0.221903,0.656920,0.520482
97,0.484060,0.999980,0.980438,2.386684,0.580847,-1.495166,0.141109,0.740765,-0.657966
98,0.913808,1.000000,-0.504775,-0.955303,0.000847,0.723666,-0.914411,0.029106,0.601174


##1. Линейная форма функции пересчёта

In [41]:
#Построение линейной регрессионной модели
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate_linear = []
  variance_linear = []
  error_linear = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [42]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1   0.069012
c11  0.285365
b11 -0.000082


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,0.927468,1.390267,0.347837,0.333565,2.037107e-06,0.014280
1,1,-0.907413,-0.362128,0.899839,-0.189902,1.187534e-02,1.090302
2,1,-0.787505,1.132473,-0.157316,-0.155807,2.276232e-08,0.001509
3,1,0.367708,2.214776,0.830017,0.173761,4.306715e-03,0.656594
4,1,-0.113440,3.139334,0.613092,0.036383,3.325939e-03,0.577007
...,...,...,...,...,...,...,...
95,1,0.596547,32.184465,-0.447428,0.236601,4.678949e-03,0.684381
96,1,0.970966,3.866204,0.984184,0.345774,4.075675e-03,0.638739
97,1,0.193689,0.493438,0.484060,0.124244,1.294676e-03,0.360002
98,1,0.972753,3.068410,0.913808,0.346350,3.220088e-03,0.567751


###Для функции $x_2[t+1]$

In [44]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.997216
b22 -0.003656


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,0.111744,0.999999,0.996807,1.018845e-07,0.003192
1,1,0.546883,0.986653,0.995216,7.332412e-07,0.008563
2,1,0.535319,0.988257,0.995259,4.903021e-07,0.007002
3,1,-0.359150,0.998927,0.998528,1.589497e-09,0.000399
4,1,0.232599,0.999921,0.996365,1.264168e-07,0.003556
...,...,...,...,...,...,...
95,1,0.289952,0.999703,0.996156,1.258294e-07,0.003547
96,1,0.593700,0.978183,0.995045,2.843238e-06,0.016862
97,1,0.185581,0.999980,0.996537,1.185007e-07,0.003442
98,1,0.020678,1.000000,0.997140,8.179677e-08,0.002860


###Для функции $x_3[t+1]$

In [46]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

           X3
a3   4.900937
c31  0.021637
c32 -5.118554
b33  0.004011


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,0.927468,0.999962,2.726109,0.198042,-0.186417,1.478087e-03,0.384519
1,1,-0.907413,0.999929,16.521043,-0.994704,-0.170616,6.791219e-03,0.824218
2,1,-0.787505,1.000000,1.101290,0.937629,-0.230238,1.363914e-02,1.168050
3,1,0.367708,0.999991,8.750406,-0.956903,-0.174515,6.121317e-03,0.782511
4,1,-0.113440,0.997203,0.041700,-0.917253,-0.205589,5.064655e-03,0.711776
...,...,...,...,...,...,...,...,...
95,1,0.596547,0.990093,-0.150859,-0.158759,-0.154602,1.727804e-07,0.004157
96,1,0.970966,0.989972,-0.573110,0.819624,-0.147577,9.354775e-03,0.967353
97,1,0.193689,0.998328,1.484246,0.980438,-0.198911,1.390864e-02,1.179534
98,1,0.972753,0.999997,2.438083,-0.504775,-0.186772,1.011258e-03,0.318053


###Для функции $y_1[t]$

In [48]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

           Y1
a1   0.112276
c11  0.553511
b11  0.000026


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,0.927468,1.390267,0.642466,0.625676,0.000003,0.002633
1,1,-0.907413,-0.362128,2.386961,-0.389996,0.077115,0.435493
2,1,-0.787505,1.132473,-0.617095,-0.323587,0.000861,0.046029
3,1,0.367708,2.214776,0.100676,0.315864,0.000463,0.033747
4,1,-0.113440,3.139334,-0.012171,0.049568,0.000038,0.009682
...,...,...,...,...,...,...,...
95,1,0.596547,32.184465,0.014316,0.443317,0.001840,0.067278
96,1,0.970966,3.866204,0.247469,0.649818,0.001619,0.063098
97,1,0.193689,0.493438,0.172752,0.219498,0.000022,0.007331
98,1,0.972753,3.068410,0.312673,0.650786,0.001143,0.053024


###Для функции $y_2[t]$

In [50]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.510330
b22  0.062375


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,0.111744,0.333934,0.517300,3.362306e-04,0.245157
1,1,0.546883,0.721129,0.544442,3.121840e-04,0.236228
2,1,0.535319,0.714224,0.543721,2.907135e-04,0.227960
3,1,-0.359150,0.592856,0.487928,1.100997e-04,0.140288
4,1,0.232599,0.480112,0.524838,2.000462e-05,0.059799
...,...,...,...,...,...,...
95,1,0.289952,0.534702,0.528416,3.952021e-07,0.008405
96,1,0.593700,0.747952,0.547362,4.023618e-04,0.268185
97,1,0.185581,0.429555,0.521906,8.528553e-05,0.123471
98,1,0.020678,0.143792,0.511620,1.352971e-03,0.491780


###Для функции $y_3[t]$

In [52]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

           Y3
a3  -5.932059
c31  3.650531
c32  5.688237
b33  0.031228


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,0.927468,0.999962,2.726109,2.298606,3.226840,0.008616,0.055688
1,1,-0.907413,0.999929,16.521043,-13.629427,-3.040850,1.121180,0.635247
2,1,-0.787505,1.000000,1.101290,-0.788428,-3.084242,0.052708,0.137734
3,1,0.367708,0.999991,8.750406,2.925107,1.371712,0.024130,0.093194
4,1,-0.113440,0.997203,0.041700,-0.004311,-0.672542,0.004465,0.040090
...,...,...,...,...,...,...,...,...
95,1,0.596547,0.990093,-0.150859,-0.082557,1.872824,0.038235,0.117310
96,1,0.970966,0.989972,-0.573110,-0.510537,3.225780,0.139601,0.224155
97,1,0.193689,0.998328,1.484246,0.261745,0.500080,0.000568,0.014299
98,1,0.972753,0.999997,2.438083,2.156055,3.383361,0.015063,0.073630


####Матрица ошибок и отклонений

In [71]:
errorsTable = pd.DataFrame([[ sum(X1_linearTable['variance']), sum(X2_linearTable['variance']), sum(X3_linearTable['variance']),
                              sum(Y1_linearTable['variance']), sum(Y2_linearTable['variance']), sum(Y3_linearTable['variance'])],
                            [ sum(X1_linearTable['error, %']), sum(X2_linearTable['error, %']), sum(X3_linearTable['error, %']),
                              sum(Y1_linearTable['error, %']), sum(Y2_linearTable['error, %']), sum(Y3_linearTable['error, %'])]],
                           index=['E', 'A, %'], columns=['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
errorsTable

,x1,x2,x3,y1,y2,y3
E,0.459564,0.000027,0.479686,2.436967,0.025210,12.712459
"A, %",59.989084,0.359492,61.282553,11.242611,17.234015,15.570089


##2. Квадратичная форма функции пересчёта

In [54]:
#Построение квадратичной регрессионной модели
def buildQuadraticRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate = []
  variance = []
  error = []

  #переменные
  Function = (model1.copy())[cols[0:(len(cols) - 1)]]
  i = 0
  name_col = ''
  while(i < len(cols) - 1):
    temporary_cols = []
    j = i

    while(j < len(cols) - 1):
      name_col = cols[i] + ' * ' + cols[j]
      for k in range(len(model1)):
        temporary_cols.append(Function[cols[i]][k] * Function[cols[j]][k])

      Function[name_col] = temporary_cols
      temporary_cols.clear()
      j += 1

    i += 1

  #значение функции
  Function[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]

  #доп. столбец
  Function.insert(0, "optional_col", vector_ones)

  A = np.array(Function)
  #A_psev = BenIsrael(A[:, 0:(len(A[0]) - 1)])
  X = solvingSLAE(A[:, 0:(len(A[0]) - 1)], np.matrix(A[:, len(A[0]) - 1]).T)

  for i in range(len(Function)):
    for j in range(0, len(X)):
      if(j == 0):
        estimate.append(X[j,0])
      else:
        estimate[i] += X[j,0] * np.array(Function)[i,j]

    variance.append((1.0 / len(Function)) * (np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])**2)
    #error_linear.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            #/ np.array(Function)[i, len(np.array(Function)[0]) - 1]) * 100)
    error.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            / max(np.array(Function)[:, len(np.array(Function)[0]) - 1])) * 100)

  Function['regression'] = estimate
  Function['variance'] = variance
  Function['error, %'] = error

  return X, Function

###Для функции $x_1[t + 1]$

In [55]:
#x1, v1+
X1_quadratic, X1_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_quadratic, columns=['X1'], index=['a1', 'c11', 'b11', 'd111', 'd113', 'd112']))
X1_quadraticTable

                X1
a1    1.505362e-01
c11   3.438239e-01
b11  -3.358476e-03
d111 -1.043876e-01
d113 -6.010875e-03
d112 -4.385088e-08


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),x1(10),regression,variance,"error, %"
0,1,0.927468,1.390267,0.860197,1.289428,1.932841,0.347837,0.367208,0.000004,0.019381
1,1,-0.907413,-0.362128,0.823398,0.328600,0.131137,0.899839,-0.248166,0.013179,1.148596
2,1,-0.787505,1.132473,0.620164,-0.891828,1.282495,-0.157316,-0.183407,0.000007,0.026105
3,1,0.367708,2.214776,0.135209,0.814390,4.905234,0.830017,0.250515,0.003358,0.579801
4,1,-0.113440,3.139334,0.012869,-0.356125,9.855417,0.613092,0.101786,0.002614,0.511569
...,...,...,...,...,...,...,...,...,...,...
95,1,0.596547,32.184465,0.355868,19.199541,1035.839792,-0.447428,0.094953,0.002942,0.542660
96,1,0.970966,3.866204,0.942775,3.753953,14.947535,0.984184,0.350414,0.004017,0.634097
97,1,0.193689,0.493438,0.037515,0.095573,0.243481,0.484060,0.210983,0.000746,0.273217
98,1,0.972753,3.068410,0.946249,2.984806,9.415141,0.913808,0.357968,0.003090,0.556126


###Для функции $x_2[t+1]$

In [57]:
#v2+
X2_quadratic, X2_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_quadratic, columns=['X2'], index=['a2', 'b22', 'd221']))
X2_quadraticTable

            X2
a2    1.002391
b22  -0.000989
d221 -0.046286


,optional_col,v2(9),v2(9) * v2(9),x2(10),regression,variance,"error, %"
0,1,0.111744,0.012487,0.999999,1.001703,2.902595e-08,0.001704
1,1,0.546883,0.299081,0.986653,0.988007,1.832577e-08,0.001354
2,1,0.535319,0.286567,0.988257,0.988598,1.165037e-09,0.000341
3,1,-0.359150,0.128989,0.998927,0.996776,4.627679e-08,0.002151
4,1,0.232599,0.054102,0.999921,0.999657,6.959011e-10,0.000264
...,...,...,...,...,...,...,...
95,1,0.289952,0.084072,0.999703,0.998213,2.219422e-08,0.001490
96,1,0.593700,0.352480,0.978183,0.985489,5.337641e-07,0.007306
97,1,0.185581,0.034440,0.999980,1.000614,4.019615e-09,0.000634
98,1,0.020678,0.000428,1.000000,1.002351,5.526925e-08,0.002351


###Для функции $x_3[t+1]$

In [59]:
#x1, x2, v3+
X3_quadratic, X3_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_quadratic, columns=['X3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336',]))
X3_quadraticTable

               X3
a3   -1326.208735
c31     25.282513
c32   2673.519068
b33      1.352781
d331     0.244825
d332   -25.381444
d333     0.009709
d334 -1347.663711
d335    -1.340365
d336    -0.001003


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),x3(10),regression,variance,"error, %"
0,1,0.927468,0.999962,2.726109,0.860197,0.927432,2.528379,0.999923,2.726005,7.431673,0.198042,-0.181724,0.001442,0.379825
1,1,-0.907413,0.999929,16.521043,0.823398,-0.907349,-14.991407,0.999859,16.519877,272.944870,-0.994704,-0.274872,0.005182,0.719946
2,1,-0.787505,1.000000,1.101290,0.620164,-0.787505,-0.867271,1.000000,1.101290,1.212839,0.937629,-0.119600,0.011177,1.057395
3,1,0.367708,0.999991,8.750406,0.135209,0.367705,3.217592,0.999983,8.750331,76.569602,-0.956903,-0.293242,0.004404,0.663765
4,1,-0.113440,0.997203,0.041700,0.012869,-0.113122,-0.004730,0.994415,0.041583,0.001739,-0.917253,-0.295981,0.003860,0.621370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.596547,0.990093,-0.150859,0.355868,0.590637,-0.089994,0.980283,-0.149364,0.022758,-0.158759,-0.096249,0.000039,0.062520
96,1,0.970966,0.989972,-0.573110,0.942775,0.961229,-0.556471,0.980044,-0.567363,0.328456,0.819624,0.091139,0.005307,0.728600
97,1,0.193689,0.998328,1.484246,0.037515,0.193365,0.287481,0.996658,1.481764,2.202987,0.980438,-0.300093,0.016398,1.280732
98,1,0.972753,0.999997,2.438083,0.946249,0.972750,2.371654,0.999993,2.438075,5.944250,-0.504775,-0.170453,0.001118,0.334374


###Для функции $y_1[t]$

In [61]:
#x1, v1+
Y1_quadratic, Y1_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_quadratic, columns=['Y1'], index=['a1', 'c11', 'b11', 'd111', 'd112', 'd113']))
Y1_quadraticTable

                Y1
a1    1.065614e-01
c11   6.162921e-01
b11  -2.635810e-03
d111  5.115614e-02
d112 -6.394814e-03
d113 -1.502166e-07


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),y1(9),regression,variance,"error, %"
0,1,0.927468,1.390267,0.860197,1.289428,1.932841,0.642466,0.710246,0.000046,0.010630
1,1,-0.907413,-0.362128,0.823398,0.328600,0.131137,2.386961,-0.411695,0.078325,0.438896
2,1,-0.787505,1.132473,0.620164,-0.891828,1.282495,-0.617095,-0.344329,0.000744,0.042776
3,1,0.367708,2.214776,0.135209,0.814390,4.905234,0.100676,0.329047,0.000522,0.035814
4,1,-0.113440,3.139334,0.012869,-0.356125,9.855417,-0.012171,0.031309,0.000019,0.006819
...,...,...,...,...,...,...,...,...,...,...
95,1,0.596547,32.184465,0.355868,19.199541,1035.839792,0.014316,0.284648,0.000731,0.042395
96,1,0.970966,3.866204,0.942775,3.753953,14.947535,0.247469,0.718990,0.002223,0.073946
97,1,0.193689,0.493438,0.037515,0.095573,0.243481,0.172752,0.225937,0.000028,0.008341
98,1,0.972753,3.068410,0.946249,2.984806,9.415141,0.312673,0.727292,0.001719,0.065022


###Для функции $y_2[t]$

In [63]:
#v2+
Y2_quadratic, Y2_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_quadratic, columns=['Y2'], index=['a2', 'b22', 'd221']))
Y2_quadraticTable

            Y2
a2    0.349922
b22  -0.020276
d221  1.434546


,optional_col,v2(9),v2(9) * v2(9),y2(9),regression,variance,"error, %"
0,1,0.111744,0.012487,0.333934,0.365569,0.000010,0.042295
1,1,0.546883,0.299081,0.721129,0.767878,0.000022,0.062503
2,1,0.535319,0.286567,0.714224,0.750161,0.000013,0.048047
3,1,-0.359150,0.128989,0.592856,0.542244,0.000026,0.067668
4,1,0.232599,0.054102,0.480112,0.422818,0.000033,0.076601
...,...,...,...,...,...,...,...
95,1,0.289952,0.084072,0.534702,0.464648,0.000049,0.093661
96,1,0.593700,0.352480,0.747952,0.843532,0.000091,0.127790
97,1,0.185581,0.034440,0.429555,0.395565,0.000012,0.045444
98,1,0.020678,0.000428,0.143792,0.350116,0.000426,0.275852


###Для функции $y_3[t]$

In [65]:
#x1, x2, v3+
Y3_quadratic, Y3_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_quadratic, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336']))
Y3_quadraticTable

              Y3
a3    284.866376
c31     3.431701
c32  -574.804234
b33     0.120643
d331   -0.003885
d332   -3.442065
d333    0.911497
d334  289.944519
d335   -0.121679
d336    0.000070


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),y3(9),regression,variance,"error, %"
0,1,0.927468,0.999962,2.726109,0.860197,0.927432,2.528379,0.999923,2.726005,7.431673,2.298606,2.295954,7.034673e-08,0.000159
1,1,-0.907413,0.999929,16.521043,0.823398,-0.907349,-14.991407,0.999859,16.519877,272.944870,-13.629427,-13.650138,4.289709e-06,0.001243
2,1,-0.787505,1.000000,1.101290,0.620164,-0.787505,-0.867271,1.000000,1.101290,1.212839,-0.788428,-0.779159,8.591383e-07,0.000556
3,1,0.367708,0.999991,8.750406,0.135209,0.367705,3.217592,0.999983,8.750331,76.569602,2.925107,2.931439,4.009297e-07,0.000380
4,1,-0.113440,0.997203,0.041700,0.012869,-0.113122,-0.004730,0.994415,0.041583,0.001739,-0.004311,-0.009599,2.796000e-07,0.000317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.596547,0.990093,-0.150859,0.355868,0.590637,-0.089994,0.980283,-0.149364,0.022758,-0.082557,-0.084532,3.901154e-08,0.000118
96,1,0.970966,0.989972,-0.573110,0.942775,0.961229,-0.556471,0.980044,-0.567363,0.328456,-0.510537,-0.502687,6.162797e-07,0.000471
97,1,0.193689,0.998328,1.484246,0.037515,0.193365,0.287481,0.996658,1.481764,2.202987,0.261745,0.258886,8.172958e-08,0.000172
98,1,0.972753,0.999997,2.438083,0.946249,0.972750,2.371654,0.999993,2.438075,5.944250,2.156055,2.152544,1.232555e-07,0.000211


####Матрица ошибок и отклонений

In [72]:
errorsTable2 = pd.DataFrame([[sum(X1_quadraticTable['variance']), sum(X2_quadraticTable['variance']), sum(X3_quadraticTable['variance']),
                              sum(Y1_quadraticTable['variance']), sum(Y2_quadraticTable['variance']), sum(Y3_quadraticTable['variance'])],
                            [ sum(X1_quadraticTable['error, %']), sum(X2_quadraticTable['error, %']), sum(X3_quadraticTable['error, %']),
                              sum(Y1_quadraticTable['error, %']), sum(Y2_quadraticTable['error, %']), sum(Y3_quadraticTable['error, %'])]],
                           index=['E', 'A, %'], columns=['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
errorsTable2

,x1,x2,x3,y1,y2,y3
E,0.440036,0.000006,0.457601,2.417912,0.005165,0.000270
"A, %",57.903807,0.189969,59.462524,11.080550,7.796161,0.061558
